# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store quantum circuits, experiments, build experiment, and excute them on local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy on local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install qurrium
```


In [1]:
from IPython.display import display
from qurry import __version__

print(f"| Current Version: {__version__}")

| Current Version: 0.10.1.dev2


## 1.1 Setup Environment and Creat experiment executor


In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit

- Import simulator


In [3]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

- Next, we will initialize our `EntropyMeasure` instance. There are two methods implemented based on two theories. The defaul method is `randomized` based on Randomized Measure. The other method is `hadamard` based on Hadamard Test. 

- In the following tutorial, we will use `randomized` as our measurement method.


In [4]:
experiment_hadamard = EntropyMeasure(method="hadamard")
print('| The first method is which is `Hadamard Test`".')
experiment_randomized = EntropyMeasure()
print("| The default method is which is `Randomized Measure` we will use.")

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuits

- In `Qurry`, there are built-in quantum circuits that construct certain wave functions, or equivalently, quantum states. They can be imported from `qurry.recipe`.
- The user can define customized circuits following Qiskit workflow.



In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘


In [7]:
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)

| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                                        └───┘


In [8]:
sample03 = QuantumCircuit(8)
sample03.x(range(0, 8, 2))
print("| Custom circuit:")
print(sample03)

| Custom circuit:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘
q_5: ─────
     ┌───┐
q_6: ┤ X ├
     └───┘
q_7: ─────
          


- After we prepare three quantum circuits, we append them to the `EntropyMeasure` object by the `.add()` method.
- The attribute `.waves` of `Qurry` object is a customized `dict`. we can check the stored circuits by printing the dictionary.

In [9]:
experiment_randomized.add(
    wave=sample01,  # The circuit object
    key="TrivialParamagnet",  # The name of the circuit
    replace=False,  # True for if you want to replace the circuit with the same name.
)  # Default is False, so if you add the same name when False,
# it will be added with serial number.
experiment_randomized.add(sample02, "GHZ")
experiment_randomized.add(sample03)
print(experiment_randomized.waves)

{'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x77ed4a2909e0>, 'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x77ed49b6a1e0>, 2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x77ed8c3770e0>}


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.


## 1.3 Excute Circuits


- We can use `.measure()` method to excute the specific circuit in the `EntropyMeasure` object. The `.measure()` method  returns a hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- `EntropyMeasure.measure()` requires the following input parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurements
  - shots: the number of shots for each measurement. The default is 1024.


In [10]:
exp1 = experiment_randomized.measure(
    wave="TrivialParamagnet", times=100, shots=1024, backend=backend_sim
)
exp1

'a3a10311-22cb-4568-a5ed-86b4b9e78cd3'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`


In [11]:
exp2 = experiment_randomized.measure(
    wave="TrivialParamagnet",
    times=100,
    shots=1024,
)
exp2

'8afa4488-54ed-4fc1-82e2-ab44ba556fd9'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.


In [12]:
print(experiment_randomized.exps)

{'a3a10311-22cb-4568-a5ed-86b4b9e78cd3': <EntropyMeasureRandomizedExperiment(exp_id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3, EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None), Commonparams(exp_id='a3a10311-22cb-4568-a5ed-86b4b9e78cd3', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x77ed8c377da0>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-12-10 18:47:06', 'run.001': '2024-12-10 18:47:06'})), unused_args_num=0, analysis_num=0)>, '8afa4488-54ed-4fc1-82e2-ab44ba556fd9': <EntropyMeasureRandomizedExperiment(exp_

- The following is the other informatiom you can access.


In [13]:
experiment_randomized.waves

WaveContainer({
  'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x77ed4a2909e0>,
  'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x77ed49b6a1e0>,
  2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x77ed8c3770e0>})

In [14]:
experiment_randomized.exps[exp1]

<EntropyMeasureRandomizedExperiment(exp_id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3, 
  EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None),
  Commonparams(exp_id='a3a10311-22cb-4568-a5ed-86b4b9e78cd3', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x77ed8c377da0>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-12-10 18:47:06', 'run.001': '2024-12-10 18:47:06'})),
  unused_args_num=0,
  analysis_num=0))>

In [15]:
print(experiment_randomized.exps[exp1].statesheet())

 # EntropyMeasureRandomizedExperiment with exp_id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3
 - arguments
   - exp_name --------------------- experiment.N_U_100.qurrent_randomized
   - times ------------------------ 100
   - qubits_measured -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - registers_mapping ------------ {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - actual_num_qubits ------------ 8
   - unitary_located -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - random_unitary_seeds --------- None
 - commonparams
   - exp_id ----------------------- a3a10311-22cb-4568-a5ed-86b4b9e78cd3   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - target_keys ------------------ ['TrivialParamagnet']
   - shots ------------------------ 1024
   - backend ---------------------- <qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x77ed8c377da0>
   - run_args --------------------- {}
   - transpile_args --------------- {}
   - tags ---------------

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.


---

## 1.4 Post-Processing

- To obtain the Renyi Entropy of the whole system and the subsystems. We use `exps[].analyze()` method to calculate and store a `Analysis` object in `Experiment`.
- The `analyze` method requires the following input parameters:
  - degree: The size or selected qubits of the subsystem. Please refer to the following chart for the selection.


### Qubit Selction Hint

For example, consider a system with 12 qubits indexed as `[0, 1, ... 11]`.

Entering an index and running `.analyze` will select the bitstring corresponding to the chosen qubit.

In [16]:
exp1_report1 = experiment_randomized.exps[exp1].analyze(selected_qubits=[3, 4])
# The subsystem of qubits 3 and 4 is selected for the analysis.
exp1_report2 = experiment_randomized.exps[exp1].analyze(selected_qubits=[0])
# The subsystem of qubit 0 is selected for the analysis.

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.


In [17]:
exp1_report2.export()

({'purity': np.float64(0.9987123489379883),
  'entropy': np.float64(0.0018588848560976375),
  'puritySD': np.float64(0.4431059443310547),
  'entropySD': np.float64(0.6400909622822005),
  'num_classical_registers': 8,
  'classical_registers': [0],
  'classical_registers_actually': [0],
  'all_system_source': "AnalysisHeader(serial=0, datetime='2024-12-10 18:47:09', summoner=None, log={})",
  'purityAllSys': np.float64(1.1991012763977051),
  'entropyAllSys': np.float64(-0.2619535142869742),
  'puritySDAllSys': np.float64(1.8081965056331681),
  'entropySDAllSys': np.float64(2.1755261069078426),
  'num_classical_registers_all_sys': 8,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
  'errorRate': np.float64(-0.09539128429840622),
  'mitigatedPurity': np.float64(0.915634472076233),
  'mitigatedEntropy': np.float64(0.1271563159635713),
  'counts_num': 100,
  'taking_time': 0.000847842,
  'taking_time_all_sys': 0.03056876,
  'counts_u

In [18]:
print(exp1_report1)
print(exp1_report1.header)

EntropyMeasureRandomizedAnalysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=1.0170664024353027, entropy=-0.02441387323355597, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-12-10 18:47:09', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.


In [19]:
print(experiment_randomized.exps[exp1].reports)
print(experiment_randomized.exps[exp1].reports[0])
print(experiment_randomized.exps[exp1].reports[0].header)

AnalysisContainer(length=2, {0{...}, 1{...}})
EntropyMeasureRandomizedAnalysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=1.0170664024353027, entropy=-0.02441387323355597, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-12-10 18:47:09', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.


In [20]:
main, side_prodct = experiment_randomized.exps[exp1].reports[0].export()

- And there is it


In [21]:
main

{'purity': np.float64(1.0170664024353027),
 'entropy': np.float64(-0.02441387323355597),
 'puritySD': np.float64(0.7149922703351819),
 'entropySD': np.float64(1.01420693891432),
 'num_classical_registers': 8,
 'classical_registers': [3, 4],
 'classical_registers_actually': [3, 4],
 'all_system_source': 'independent',
 'purityAllSys': np.float64(1.1991012763977051),
 'entropyAllSys': np.float64(-0.2619535142869742),
 'puritySDAllSys': np.float64(1.8081965056331681),
 'entropySDAllSys': np.float64(2.1755261069078426),
 'num_classical_registers_all_sys': 8,
 'classical_registers_all_sys': None,
 'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
 'errorRate': np.float64(-0.09539128429840622),
 'mitigatedPurity': np.float64(0.8892848300278511),
 'mitigatedEntropy': np.float64(0.1692825194159182),
 'counts_num': 100,
 'taking_time': 0.001836907,
 'taking_time_all_sys': 0.03056876,
 'counts_used': None,
 'input': {'num_qubits': 8,
  'selected_qubits': [3, 4],
  'registers_mapp

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.


In [22]:
side_prodct

{'purityCells': {72: 1.01885986328125,
  64: 0.47210121154785156,
  29: 0.5712013244628906,
  88: 0.7247982025146484,
  43: 0.33567047119140625,
  48: 0.7885761260986328,
  2: 0.6254806518554688,
  7: 0.2975730895996094,
  63: 1.0704784393310547,
  35: 2.2328662872314453,
  68: 0.5042533874511719,
  74: 0.3648185729980469,
  14: 2.6865673065185547,
  51: 1.7734661102294922,
  21: 0.43605804443359375,
  5: 0.9750347137451172,
  66: 0.6367073059082031,
  23: 0.3985443115234375,
  82: 2.5035934448242188,
  11: 1.4542160034179688,
  1: 0.780731201171875,
  58: 1.1535968780517578,
  37: 3.1112632751464844,
  55: 2.5993518829345703,
  69: 0.6573581695556641,
  60: 0.835113525390625,
  36: 0.7298221588134766,
  17: 0.386505126953125,
  70: 0.5398387908935547,
  40: 0.6218757629394531,
  91: 0.9060096740722656,
  81: 1.0071353912353516,
  76: 1.9485435485839844,
  93: 1.19097900390625,
  61: 0.6706390380859375,
  24: 0.90240478515625,
  77: 0.8952980041503906,
  89: 0.9688892364501953,
  99: 0

In [23]:
experiment_randomized.exps[exp1].reports[0].statesheet()

 # EntropyMeasureRandomizedAnalysis with serial=0
 - header
   - serial -------------------------------------- 0
   - datetime ------------------------------------ 2024-12-10 18:47:09
   - summoner ------------------------------------ None
   - log ----------------------------------------- {}
 - input
   - num_qubits ---------------------------------- 8
   - selected_qubits ----------------------------- [3, 4]
   - registers_mapping --------------------------- {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - shots --------------------------------------- 1024
   - unitary_located ----------------------------- [0, 1, 2, 3, 4, 5, 6, 7]
 - outfields ----------------------------------- 0 ........  # Number of unused arguments.
 - content
   - purity -------------------------------------- 1.0170664024353027
   - entropy ------------------------------------- -0.02441387323355597
   - puritySD ------------------------------------ 0.7149922703351819
   - entropySD -------------------------

---

## 1.5 Export Current Result

Also, Qurry provides a method to export all results as multiple files in a folder.
Just use the method `write` of each `Experiment` instance.

In [24]:
experiment_randomized.exps[exp1].write()

('a3a10311-22cb-4568-a5ed-86b4b9e78cd3',
 {'folder': 'experiment.N_U_100.qurrent_randomized.003',
  'qurryinfo': 'experiment.N_U_100.qurrent_randomized.003/qurryinfo.json',
  'args': 'experiment.N_U_100.qurrent_randomized.003/args/experiment.N_U_100.qurrent_randomized.003.id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3.args.json',
  'advent': 'experiment.N_U_100.qurrent_randomized.003/advent/experiment.N_U_100.qurrent_randomized.003.id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3.advent.json',
  'legacy': 'experiment.N_U_100.qurrent_randomized.003/legacy/experiment.N_U_100.qurrent_randomized.003.id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3.legacy.json',
  'tales.unitaryOP': 'experiment.N_U_100.qurrent_randomized.003/tales/experiment.N_U_100.qurrent_randomized.003.id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3.unitaryOP.json',
  'tales.randomized': 'experiment.N_U_100.qurrent_randomized.003/tales/experiment.N_U_100.qurrent_randomized.003.id=a3a10311-22cb-4568-a5ed-86b4b9e78cd3.randomized.json',
  'reports': 'exper

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [25]:
from qurry.process import AVAIBILITY_STATESHEET

print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.10.1.dev2
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes   
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes   
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes   
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes   
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct ..

In [26]:
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.10.1.dev2
--------------------------------------------
 ### Qiskit version
 - main
   - qiskit-aer .............. 0.15.1
   - qiskit-ibm-runtime ...... 0.34.0
 - deprecated
 - into-community
--------------------------------------------

